In [1]:
import json
import pandas as pd
import time

In [2]:
# https://stackoverflow.com/questions/51419785/extract-data-from-wikidata-in-python


def get_candidates(candidate_entities):
    API_ENDPOINT = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'language': 'en',
        'search': ''
    }
    
    candidates_sent = []
    for label in candidate_entities:
        candidates_for_one = []
        if len(label) ==0:
            continue
        
        params['search'] = str(label)  # set search from the params to our query word
        try:
            r = requests.get(API_ENDPOINT, params = params).json()
        except:
            return "API failed  " + label
#             return label


        try:
            search_arr = r["search"]
        except:
            print (label)
            return label

        for c in search_arr:
            c_in =[]
            try:
                c_in.append(c["id"])
                c_in.append(c["label"])
                c_in.append(c["description"])
                candidates_for_one.append(c_in)
            except:
                pass

        candidates_sent.append(candidates_for_one)
            
    return candidates_sent

In [3]:
def get_labels(ent_id):
    labels = []
    API_ENDPOINT = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbgetentities',
        'format': 'json',
        'languages': 'en',
        'props': 'labels|aliases',    
        'ids': ''
    }

        
    params['ids'] = str(ent_id)
    try:
        response = requests.get(API_ENDPOINT, params = params).json()['entities']
        labels.append(response[str(ent_id)]['labels']['en']['value'])
        
        for alias in response[str(ent_id)]['aliases']['en']:
            labels.append(alias['value'])
    except:
        return (ent_id, labels)
    return (ent_id, labels)

In [4]:
# jupyter notebook never finishes processing using multiprocessing python 3
# https://stackoverflow.com/questions/47313732/jupyter-notebook-never-finishes-processing-using-multiprocessing-python-3

from multiprocessing import Pool
from functools import partial
import inspect
import time
import os
import requests


def parallal_task(func, iterable, *params): 

    with open(f'./tmp_func.py', 'w') as file:
        file.write("import requests \n")
        file.write(inspect.getsource(func).replace(func.__name__, 'task'))

    from tmp_func import task
    pool = Pool(processes=10)
    res = pool.map(task, iterable)
    pool.close()
    
    os.remove('./tmp_func.py')
    return res
#     else:
#         raise "Not in Jupyter Notebook"

In [5]:
from multiprocessing import Pool
from functools import partial
import inspect
import time
import os
import requests


def parallal_task_2(func, iterable, *params): 

    with open(f'./tmp_func_2.py', 'w') as file:
        file.write("import requests \n")
        file.write(inspect.getsource(func).replace(func.__name__, 'task'))

    from tmp_func_2 import task
    pool = Pool(processes=10)
    res = pool.map(task, iterable)
    pool.close()
    
    os.remove('./tmp_func_2.py')
    return res

In [6]:
from fuzzywuzzy import fuzz

# when creating a function the arguments are (questions and the results wikidata api after lower())
def find_nearest_word_in_question(questions, lower_wikidata_replies):
    entities_list = []
    found_match = False
    counter = 0
    for i in range(len(questions)):
        found_match = False
        for label in lower_wikidata_replies[i][1]:
            if label in questions[i].lower(): 
                found_match = True
                index = questions[i].lower().find(label)
                entities_list.append(questions[i][index : len(label)+index])
                break
        if not found_match:
            split_tokens = questions[i][1].split()
            list_of_comb = []
            list_of_comb.extend(split_tokens)
            scores = []
            for j in range(len(split_tokens)-1):
                list_of_comb.append(split_tokens[j] + ' ' + split_tokens[j+1])
            for j in range(len(split_tokens)-2):
                list_of_comb.append(split_tokens[j] + ' ' + split_tokens[j+1] + ' ' + split_tokens[j+2])
            for j in range(len(split_tokens)-3):
                list_of_comb.append(split_tokens[j] + ' ' + split_tokens[j+1] + ' ' + split_tokens[j+2] + ' ' + split_tokens[j+3])

            for label in lower_wikidata_replies[i][1]:
                for token in list_of_comb: 
                    scores.append((fuzz.ratio(label, token.lower()), token, label))
            scores = sorted(scores, key=lambda x: x[0], reverse=True)

            if len(scores) and scores[0][0] < 85:
                counter += 1
                entities_list.append('')
            elif not(len(scores)):
                counter += 1
                entities_list.append('')
            elif scores[0][0] >= 85:
                entities_list.append(scores[0][1])
    return (entities_list, counter)

In [7]:
# from fuzzywuzzy import fuzz

# # when creating a function the arguments are (questions and the results wikidata api after lower())

# entities_list = []
# found_match = False
# counter = 0
# for i in range(len(questions)):
#     found_match = False
#     for label in lower_results[i][1]:
#         if label in questions[i].lower(): 
#             found_match = True
#             index = questions[i].lower().find(label)
#             entities_list.append(questions[i][index : len(label)+index])
#             break
#     if not found_match:
#         split_tokens = questions[i][1].split()
#         list_of_comb = []
#         list_of_comb.extend(split_tokens)
#         scores = []
#         for j in range(len(split_tokens)-1):
#             list_of_comb.append(split_tokens[j] + ' ' + split_tokens[j+1])
#         for j in range(len(split_tokens)-2):
#             list_of_comb.append(split_tokens[j] + ' ' + split_tokens[j+1] + ' ' + split_tokens[j+2])
#         for j in range(len(split_tokens)-3):
#             list_of_comb.append(split_tokens[j] + ' ' + split_tokens[j+1] + ' ' + split_tokens[j+2] + ' ' + split_tokens[j+3])
        
#         for label in lower_results[i][1]:
#             for token in list_of_comb: 
#                 scores.append((fuzz.ratio(label, token.lower()), token, label))
#         scores = sorted(scores, key=lambda x: x[0], reverse=True)

#         if len(scores) and scores[0][0] < 85:
#             counter += 1
#             entities_list.append('')
#         elif not(len(scores)):
#             counter += 1
#             entities_list.append('')
#         elif scores[0][0] >= 85:
#             entities_list.append(scores[0][1])

### Training Data

In [9]:
with open('2-NEL_Data/0-raw_data/simple_questions_v2/annotated_wd_data_train.txt', encoding='utf-8') as file:
    lines = file.readlines()

# print(len(lines))
training_data = []
for line in lines:
    line = line.strip().split('\t', 3)
    del line[2]
    del line[1]
    training_data.append(line)
    
qids = [line[0] for line in training_data]
questions = [line[1] for line in training_data]
# print(len(training_data))
# print(training_data[0])
# print(qids[0])
# print(questions[0])

In [10]:
start = time.time()
results = parallal_task(get_labels, qids)
print(time.time() - start)
len(results)

927.1629462242126


34374

In [11]:
results[:2]

[('Q126399',
  ['Warner Bros.',
   'Warner Bros. Pictures',
   'Warner Bros. Entertainment Inc.',
   'Warner Bros. Studios',
   'Warner Brothers',
   'WB',
   'Warner Bros. Entertainment, Inc.']),
 ('Q12439',
  ['Detroit',
   'Motor City',
   'Detroit, Michigan',
   'Detroiit',
   'Detroit, MI',
   'The D',
   'City of Detroit'])]

In [12]:
counter_empty = 0
for entity in results:
    if not(len(entity)):
        counter_empty += 1
print(counter_empty)

0


In [13]:
lower_results = []
for result in results:
    results_for_one = [label.lower() for label in result[1]]
    lower_results.append((result[0], results_for_one))
lower_results[:2]

[('Q126399',
  ['warner bros.',
   'warner bros. pictures',
   'warner bros. entertainment inc.',
   'warner bros. studios',
   'warner brothers',
   'wb',
   'warner bros. entertainment, inc.']),
 ('Q12439',
  ['detroit',
   'motor city',
   'detroit, michigan',
   'detroiit',
   'detroit, mi',
   'the d',
   'city of detroit'])]

In [14]:
(entities_list, counter) = find_nearest_word_in_question(questions, lower_results)

In [15]:
# print(entities_list[:10])
# print(counter)
# print(counter*100/len(questions))

In [16]:
if type(entities_list[0])==str:
    entities_list = [[str(entity)] for entity in entities_list]
entities_list[:10]

[['warner bros.'],
 ['detroit'],
 ['rough house rosie'],
 ['mera shikar'],
 ['chicago'],
 ['midfielder'],
 ['great patriotic war'],
 ['mike twellman'],
 ['musical film'],
 ['ellen swallow richards']]

In [17]:
start = time.time()
results_2 = parallal_task_2(get_candidates, entities_list)
print(time.time() - start)

960.2558364868164


In [18]:
df = pd.DataFrame(list(zip(qids, questions, entities_list, results_2)),columns=['qid', 'question', 'entity', 'wikidata_reply'])
df

,qid,question,entity,wikidata_reply
0,Q126399,what movie is produced by warner bros.,[warner bros.],"[[[Q126399, Warner Bros., American producer of..."
1,Q12439,who is a musician born in detroit,[detroit],"[[[Q12439, Detroit, city in and county seat of..."
2,Q7370831,who produced the film rough house rosie,[rough house rosie],"[[[Q4421558, Rough House Rosie, 1927 film by F..."
3,Q6817891,what is the language in which mera shikar was ...,[mera shikar],"[[[Q6817891, Mera Shikar, 1988 film by Keshu R..."
4,Q1297,Whats the name of a battle that happened in ch...,[chicago],"[[[Q1297, Chicago, city and county seat of Coo..."
...,...,...,...,...
34369,Q16093542,What is the nationality of anthony bailey,[anthony bailey],"[[[Q4772057, Anthony Bailey, Public relations ..."
34370,Q325741,who was homi k. bhabha especially influenced by,[homi k. bhabha],"[[[Q325741, Homi K. Bhabha, Indian critical th..."
34371,Q1062702,which artist composes video game music,[video game music],"[[[Q1062702, video game music, soundtrack or b..."
34372,Q926822,What gender is gastón filgueira,[gastón filgueira],"[[[Q926822, Gastón Filgueira, Uruguayan footba..."


In [19]:
df.to_csv('./2-NEL_Data/1-csv_format_1/simple_questions_training_dataset.csv', index=False)

### validation set

In [20]:
with open('2-NEL_Data/0-raw_data/simple_questions_v2/annotated_wd_data_valid.txt', encoding='utf-8') as file:
    lines = file.readlines()

# print(len(lines))
validation_data = []
for line in lines:
    line = line.strip().split('\t', 3)
    del line[2]
    del line[1]
    validation_data.append(line)
    
qids = [line[0] for line in validation_data]
questions = [line[1] for line in validation_data]
print(len(validation_data))
print(validation_data[0])
print(qids[0])
print(questions[0])

4867
['Q3541144', 'Who was the trump ocean club international hotel and tower named after']
Q3541144
Who was the trump ocean club international hotel and tower named after


In [21]:
start = time.time()
results = parallal_task(get_labels, qids)
print(time.time() - start)
len(results)

113.58449006080627


4867

In [22]:
results[:2]

[('Q3541144',
  ['JW Marriott Panama',
   'Trump International Hotel & Tower Panama',
   'Trump Ocean Club',
   'Trump Ocean Club International Hotel and Tower',
   'JW Marriott Panama']),
 ('Q318926',
  ['Sasha Vujačić', 'Saša Vujačić', 'Sasha Vujacic', 'Aleksander Vujačić'])]

In [23]:
counter_empty = 0
for entity in results:
    if not(len(entity)):
        counter_empty += 1
print(counter_empty)

0


In [24]:
lower_results = []
for result in results:
    results_for_one = [label.lower() for label in result[1]]
    lower_results.append((result[0], results_for_one))
lower_results[:2]

[('Q3541144',
  ['jw marriott panama',
   'trump international hotel & tower panama',
   'trump ocean club',
   'trump ocean club international hotel and tower',
   'jw marriott panama']),
 ('Q318926',
  ['sasha vujačić', 'saša vujačić', 'sasha vujacic', 'aleksander vujačić'])]

In [25]:
(valid_entities_list, counter) = find_nearest_word_in_question(questions, lower_results)

In [26]:
counter*100/len(questions)

5.855763303883296

In [27]:
if type(valid_entities_list[0])==str:
    valid_entities_list = [[str(entity)] for entity in valid_entities_list]
valid_entities_list[:10]

[['trump ocean club'],
 ['sasha vujačić'],
 ['wiebke von carolsfeld'],
 ['Seymour Parker Gilbert'],
 ['antoine de févin'],
 ['jamie hewlett'],
 ['ghost house'],
 ['yamakinkarudu'],
 ['sub-saharan africa'],
 ['fritz leiber']]

In [28]:
start = time.time()
results_2 = parallal_task_2(get_candidates, valid_entities_list)
print(time.time() - start)

118.00032162666321


In [29]:
df = pd.DataFrame(list(zip(qids, questions, valid_entities_list, results_2)),columns=['qid', 'question', 'entity', 'wikidata_reply'])
df

,qid,question,entity,wikidata_reply
0,Q3541144,Who was the trump ocean club international hot...,[trump ocean club],"[[[Q3541144, JW Marriott Panama, hotel in Pana..."
1,Q318926,where was sasha vujačić born,[sasha vujačić],"[[[Q318926, Sasha Vujačić, Slovenian basketbal..."
2,Q2568216,What is a film directed by wiebke von carolsfeld?,[wiebke von carolsfeld],"[[[Q2568216, Wiebke Carolsfeld, German film ed..."
3,Q2275923,What was Seymour Parker Gilbert's profession?,[Seymour Parker Gilbert],"[[[Q2275923, Seymour Parker Gilbert, American ..."
4,Q2856873,in what french city did antoine de févin die,[antoine de févin],"[[[Q2856873, Antoine de Févin, French composer]]]"
...,...,...,...,...
4862,Q3709870,which position did herby fortunat play in foot...,[herby fortunat],"[[[Q3709870, Herby Fortunat, Haitian-French fo..."
4863,Q150804,what kind of film is the pianist (2002 film),[the pianist],"[[[Q150804, The Pianist, 2002 biographic film ..."
4864,Q1343857,where was jon seda given birth,[jon seda],"[[[Q1343857, Jon Seda, American actor]]]"
4865,Q2981169,What is nikolaj frobenius's profession?,[nikolaj frobenius],"[[[Q2981169, Nikolaj Frobenius, Norwegian writ..."


In [30]:
df.to_csv('./2-NEL_Data/1-csv_format_1/simple_questions_val_dataset.csv', index=False)

### test data

In [31]:
with open('2-NEL_Data/0-raw_data/simple_questions_v2/annotated_wd_data_test.txt', encoding='utf-8') as file:
    lines = file.readlines()

# print(len(lines))
test_data = []
for line in lines:
    line = line.strip().split('\t', 3)
    del line[2]
    del line[1]
    test_data.append(line)
    
qids = [line[0] for line in test_data]
questions = [line[1] for line in test_data]
print(len(test_data))
print(test_data[0])
print(qids[0])
print(questions[0])

9961
['Q5487302', 'Which genre of album is harder.....faster?']
Q5487302
Which genre of album is harder.....faster?


In [32]:
start = time.time()
results = parallal_task(get_labels, qids)
print(time.time() - start)
len(results)

233.62765264511108


9961

In [33]:
results[:2]

[('Q5487302', ['Harder ... Faster']), ('Q16330302', ['Alex Golfis'])]

In [34]:
counter_empty = 0
for entity in results:
    if not(len(entity)):
        counter_empty += 1
print(counter_empty)

0


In [35]:
lower_results = []
for result in results:
    results_for_one = [label.lower() for label in result[1]]
    lower_results.append((result[0], results_for_one))
lower_results[:2]

[('Q5487302', ['harder ... faster']), ('Q16330302', ['alex golfis'])]

In [36]:
(test_entities_list, counter) = find_nearest_word_in_question(questions, lower_results)

In [37]:
counter*100/len(questions)

5.722317036442124

In [38]:
if type(test_entities_list[0])==str:
    test_entities_list = [[str(entity)] for entity in test_entities_list]
test_entities_list[:10]

[[''],
 ['alex golfis'],
 ['phil hay'],
 ['roger marquis'],
 ['yves klein'],
 ['dublin'],
 [''],
 ['akbar tandjung'],
 ['carlos gomez'],
 ['engelbert zaschka']]

In [39]:
start = time.time()
results_2 = parallal_task_2(get_candidates, test_entities_list)
print(time.time() - start)

374.175009727478


In [40]:
df = pd.DataFrame(list(zip(qids, questions, test_entities_list, results_2)),columns=['qid', 'question', 'entity', 'wikidata_reply'])
df

,qid,question,entity,wikidata_reply
0,Q5487302,Which genre of album is harder.....faster?,[],[]
1,Q16330302,what city was alex golfis born in,[alex golfis],"[[[Q16330302, Alex Golfis, Greek actor (1948-2..."
2,Q16225521,what film is by the writer phil hay?,[phil hay],"[[[Q16225521, Phil Hay, screenwriter], [Q71819..."
3,Q7358590,Where did roger marquis die,[roger marquis],"[[[Q7358592, Roger Marquis, 2nd Earl of Woolto..."
4,Q154335,what was the cause of death of yves klein,[yves klein],"[[[Q154335, Yves Klein, 20th century French vi..."
...,...,...,...,...
9956,Q1447249,who was the creator of the fictional character...,[doctor faustus],"[[[Q332885, Faust, protagonist of a classic Ge..."
9957,Q34863,what's a college sporting event that took plac...,[oklahoma city],"[[[Q34863, Oklahoma City, city in Oklahoma, Cl..."
9958,Q582715,what celestial object is 2974 holden,[2974 holden],"[[[Q582715, 2974 Holden, asteroid]]]"
9959,Q582147,what is the film genre for snow falling on ced...,[snow falling on cedars],"[[[Q582147, Snow Falling on Cedars, 1999 film ..."


In [41]:
df.to_csv('./2-NEL_Data/1-csv_format_1/simple_questions_test_dataset.csv', index=False)

## wikidata-disambig

In [8]:
import json

In [17]:
# Opening JSON file
f = open('2-NEL_Data/0-raw_data/wikidata-disambig/wikidata-disambig-train.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [23]:
data[0]

{'correct_id': 'Q534153',
 'string': 'captain marvel',
 'text': ' fantasy novelist David Gemmell. Achilles is featured heavily in the novel The Firebrand by Marion Zimmer Bradley. The comic book hero Captain Marvel is endowed with the courage of Achilles, as well ',
 'wrong_id': 'Q41421'}

In [26]:
entities_disamb = []
questions_disamb = []
qids_disamb = []
counter = 0

for question in data:
#     if not question['main_entity']:
#         counter += 1
#         continue
    entities_disamb.append(question['string'])
    qids_disamb.append(question['correct_id'])
    questions_disamb.append(question['text'])
    #     print(question['main_entity_text'])

In [28]:
entities_disamb[0]

'captain marvel'

# the other dataset

In [13]:
import json
# json.load('2-NEL_Data/webqsp/webqsp.train.entities.json')

In [14]:
with open('2-NEL_Data/0-raw_data/webqsp/webqsp.train.entities.json') as file:
    file_lines = file.readlines()

In [10]:
type(file_lines[0])

str

In [12]:
stud_obj = json.loads(file_lines[0])
# print(stud_obj)

In [13]:
len(stud_obj)

3098

In [14]:
stud_obj[0]

{'question_id': 'WebQTrn-0',
 'utterance': 'what is the name of justin bieber brother?',
 'entities_fb': ['m.05zppz', 'm.06w2sn5'],
 'entities': ['Q34086', 'Q44148'],
 'main_entity_text': 'Justin Bieber',
 'main_entity': 'Q34086',
 'main_entity_fb': 'm.06w2sn5',
 'main_entity_tokens': 'justin bieber',
 'main_entity_pos': [20, 33]}

In [15]:
stud_obj[1]

{'question_id': 'WebQTrn-1',
 'utterance': 'what character did natalie portman play in star wars?',
 'entities_fb': ['m.0ddt_', 'm.09l3p'],
 'entities': ['Q165713', 'Q37876'],
 'main_entity_text': 'Natalie Portman',
 'main_entity': 'Q37876',
 'main_entity_fb': 'm.09l3p',
 'main_entity_tokens': 'natalie portman',
 'main_entity_pos': [19, 34]}

In [16]:
entities_2 = []
questions_2 = []
qids = []
counter = 0

for question in stud_obj:
    if not question['main_entity']:
        counter += 1
        continue
    entities_2.append(question['main_entity_tokens'])
    qids.append(question['main_entity'])
    questions_2.append(question['utterance'])
    #     print(question['main_entity_text'])

In [17]:
print(len(stud_obj))
print(counter)
print(len(entities_2))
print(len(questions_2))

3098
108
2990
2990


In [18]:
entities_2

['justin bieber',
 'natalie portman',
 'grand bahama island',
 'bahamas',
 'john noble',
 'joakim noah',
 'redskins',
 'saki',
 'sacha baron cohen',
 'cleveland ohio',
 'draco',
 'us',
 'rome italy',
 'nina dobrev',
 'kennedy',
 'beverly cleary',
 'bali',
 'ethiopia',
 'sedona arizona',
 'bill clinton',
 'russia',
 'taylor lautner',
 'lebron james',
 'king henry',
 'malcolm x',
 'netherlands',
 'brazil',
 'france',
 'esther',
 'toronto maple leafs',
 'saint nicholas',
 'cher',
 'andrew jackson',
 'hampton va',
 'harry potter',
 'william shakespeare',
 'heathrow airport',
 'josh hutcherson',
 'san francisco giants',
 'william jennings bryan',
 'frederick douglass',
 'queen victoria',
 'sally field',
 'sherlock holmes',
 'portuguese',
 'walt disney',
 'bush',
 'tupac',
 'australia',
 'maggie grace',
 'louis armstrong',
 'joey lawrence',
 'denver co',
 '90210',
 'sparta',
 'world series',
 'matt lauer',
 'london england',
 'joan miro',
 'vincent van gogh',
 'new york knicks',
 'jack danie

In [19]:
entities_2 = [[entity] for entity in entities_2]
entities_2

[['justin bieber'],
 ['natalie portman'],
 ['grand bahama island'],
 ['bahamas'],
 ['john noble'],
 ['joakim noah'],
 ['redskins'],
 ['saki'],
 ['sacha baron cohen'],
 ['cleveland ohio'],
 ['draco'],
 ['us'],
 ['rome italy'],
 ['nina dobrev'],
 ['kennedy'],
 ['beverly cleary'],
 ['bali'],
 ['ethiopia'],
 ['sedona arizona'],
 ['bill clinton'],
 ['russia'],
 ['taylor lautner'],
 ['lebron james'],
 ['king henry'],
 ['malcolm x'],
 ['netherlands'],
 ['brazil'],
 ['france'],
 ['esther'],
 ['toronto maple leafs'],
 ['saint nicholas'],
 ['cher'],
 ['andrew jackson'],
 ['hampton va'],
 ['harry potter'],
 ['william shakespeare'],
 ['heathrow airport'],
 ['josh hutcherson'],
 ['san francisco giants'],
 ['william jennings bryan'],
 ['frederick douglass'],
 ['queen victoria'],
 ['sally field'],
 ['sherlock holmes'],
 ['portuguese'],
 ['walt disney'],
 ['bush'],
 ['tupac'],
 ['australia'],
 ['maggie grace'],
 ['louis armstrong'],
 ['joey lawrence'],
 ['denver co'],
 ['90210'],
 ['sparta'],
 ['world

In [20]:
# get_candidates([entities_2[0]])

In [21]:
get_candidates(['justin bieber'])

[[['Q34086', 'Justin Bieber', 'Canadian singer (born 1994)'],
  ['Q110037270', 'Justin Bieber', 'episode of Saturday Night Live (S38 E13)'],
  ['Q949429',
   'Justin Bieber: Never Say Never',
   '2011 3-D concert film centering on singer Justin Bieber directed by Jon M. Chu'],
  ['Q1164777', 'Justin Bieber discography', 'Wikimedia artist discography'],
  ['Q60456247',
   "Justin Bieber's albums in chronological order",
   'Wikimedia albums discography'],
  ['Q15285106',
   "Justin Bieber's Believe",
   '2013 American biographical concert film directed by Jon M. Chu'],
  ['Q60457386',
   'Justin Bieber studio albums discography',
   'albums discography']]]

In [22]:
start = time.time()
results_2 = parallal_task(get_candidates, entities_2)
print(time.time() - start)

77.55953788757324


In [23]:
results_2

[[[['Q34086', 'Justin Bieber', 'Canadian singer (born 1994)'],
   ['Q110037270', 'Justin Bieber', 'episode of Saturday Night Live (S38 E13)'],
   ['Q949429',
    'Justin Bieber: Never Say Never',
    '2011 3-D concert film centering on singer Justin Bieber directed by Jon M. Chu'],
   ['Q1164777', 'Justin Bieber discography', 'Wikimedia artist discography'],
   ['Q60456247',
    "Justin Bieber's albums in chronological order",
    'Wikimedia albums discography'],
   ['Q15285106',
    "Justin Bieber's Believe",
    '2013 American biographical concert film directed by Jon M. Chu'],
   ['Q60457386',
    'Justin Bieber studio albums discography',
    'albums discography']]],
 [[['Q37876', 'Natalie Portman', 'American-Israeli actress and filmmaker'],
   ['Q110037498',
    'Natalie Portman/Dua Lipa',
    'episode of Saturday Night Live (S43 E13)'],
   ['Q15957227',
    'Natalie Portman filmography',
    'Filmography of Natalie Portman'],
   ['Q110036707',
    'Natalie Portman/Fall Out Boy',


In [24]:
df2 = pd.DataFrame(list(zip(qids, questions_2, entities_2, results_2)),columns=['qid', 'question', 'entity', 'wikidata_reply'])
df2

,qid,question,entity,wikidata_reply
0,Q34086,what is the name of justin bieber brother?,[justin bieber],"[[[Q34086, Justin Bieber, Canadian singer (bor..."
1,Q37876,what character did natalie portman play in sta...,[natalie portman],"[[[Q37876, Natalie Portman, American-Israeli a..."
2,Q866345,what country is the grand bahama island in?,[grand bahama island],"[[[Q41171443, Gulfstream International Airline..."
3,Q778,what kind of money to take to bahamas?,[bahamas],"[[[Q778, The Bahamas, island sovereign state i..."
4,Q312399,what character did john noble play in lord of ...,[john noble],"[[[Q312399, John Noble, Australian actor and d..."
...,...,...,...,...
2985,Q928,what type of government is the philippines?,[philippines],"[[[Q928, Philippines, sovereign state in South..."
2986,Q159,what do you call russian currency?,[russian],"[[[Q7737, Russian, East Slavic language], [Q49..."
2987,Q313591,where did sir donald bradman live?,[sir donald bradman],"[[[Q313591, Don Bradmam, Australian cricketer ..."
2988,Q223507,what is the name of the broncos mascot?,[broncos],"[[[Q223507, Denver Broncos, National Football ..."


In [25]:
df2.to_csv('./2-NEL_Data/1-csv_format_1/webqsp_dataset.csv', index=False)